In [26]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [27]:
# This line is necessary to show matplotlib plots inside the jupyter notebook
%matplotlib inline

In [28]:
# import pandas
from pandas import read_table
# read csv file
items = read_table('u.item',header=None,sep='|',encoding='ISO-8859-1')
# remove collumns 2-24
items.drop(range(2,6),axis=1, inplace=True)
# name the columns
items.columns = ['itemid','title']
# show the first 5 rows of the dataframe
items.ix[0].title

C:\Users\Elvis\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead.
  after removing the cwd from sys.path.
C:\Users\Elvis\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


'Nombre_Videojuego_1'

In [29]:
# show the name of the 25-th movie (remember, in python the index always starts from 0)
#items.iloc[24]['name']

In [30]:
data = np.genfromtxt('u.data',dtype="i8,i8,i8",names=['videojuego','usuario','rating'],delimiter=',')

In [31]:
def cofiCostFunc (parameters, Y, R, n_users, n_videogames, n_features, lamb):
    cost=0
    gradient=np.zeros_like(parameters)
    
    X=np.reshape(parameters[0: n_videogames * n_features], (n_videogames, n_features) )
    Theta=np.reshape(parameters[n_videogames * n_features : parameters.shape[0]] , (n_users,n_features))
    J=(1/2)*sum(sum(np.power((X.dot(Theta.T)-Y),2)*R))   
    cost=J + ((lamb/2) * sum(sum(np.power(X,2))))+((lamb/2) * sum(sum(np.power(Theta, 2))))
    
    X_gradient = (((X.dot(Theta.T))- Y) *R).dot(Theta) + X.dot(lamb)               
    Theta_gradient = (((X.dot(Theta.T))-Y) * R).T.dot(X) + Theta.dot(lamb)
    gradient=np.append(X_gradient,Theta_gradient)
    
    return (cost, gradient)

In [32]:
Y=np.load('guardarY.npy')
R=np.load('guardarR.npy')
# Get dimensions: n_users and n_movies
n_videogames=Y.shape[0];
n_users=Y.shape[1];
n_features = 100

In [33]:
X=np.random.uniform(low=0,high=1,size=(n_videogames*n_features))
Theta=np.random.uniform(low=0,high=1,size=(n_users*n_features))
initial_parameters=np.append(X,Theta)

In [34]:
usuarioNuevo=np.zeros((n_videogames, 1))
usuarioParaRating=np.zeros((n_videogames, 1))

In [35]:
usuarioNuevoVideojuegosJugados=[1,2,3,4,5,6,7,8,9,10]
usuarioNuevoRating=[1,3,4,2,4,3,2,3,5,5]

In [36]:
for i in range(0, len(usuarioNuevoVideojuegosJugados)):
    usuarioNuevo[usuarioNuevoVideojuegosJugados[i]], usuarioParaRating[usuarioNuevoVideojuegosJugados[i]]=usuarioNuevoRating[i],1

In [37]:
Y=np.append(Y, usuarioNuevo, axis=1)
R=np.append(R, usuarioParaRating, axis=1)
n_videogames=Y.shape[0]
n_users=Y.shape[1]

In [38]:
X=np.random.uniform(low=0,high=1,size=(n_videogames*n_features))
Theta=np.random.uniform(low=0,high=1,size=(n_users*n_features))
initial_parameters=np.append(X,Theta)

In [39]:
lamb = 1.5
def cofiCostFunc_minimize(parameters):
    return cofiCostFunc(parameters,Y, R, n_users, n_videogames, n_features,lamb)
max_iter=100

In [40]:
# Minimize the function using minimize from the package scipy.optimize and get the optimized parameters
parameters = (minimize(cofiCostFunc_minimize,initial_parameters,method="CG",jac=True,
                   options={'maxiter':max_iter, "disp":True})).x

         Current function value: 17961.545171
         Iterations: 100
         Function evaluations: 152
         Gradient evaluations: 152


In [41]:
vectX=parameters[0:((n_videogames*n_features))];
vectT=parameters[(n_videogames*n_features):(len(parameters))];  
X=vectX.reshape(n_videogames,n_features);
Theta=vectT.reshape(n_users,n_features);


#Multiplicación matricial de la X y la traspuesta de la T
predicciones=np.dot(X,Theta.T);

In [42]:
indiceUsuarioNuevo=len(Y[0])-1
videojuegosNoJugadosUsuarioNuevo=np.where(R[:, indiceUsuarioNuevo]==0)
top10videojuegos=np.fliplr(np.sort(predicciones[videojuegosNoJugadosUsuarioNuevo,indiceUsuarioNuevo]))
posicionesMejoresJuegos=list()
top10UsuarioNuevo=top10videojuegos[0][0:10]

for i in range(0, len(top10UsuarioNuevo)):
    valoracion=np.where(predicciones==top10UsuarioNuevo[i])
    posicionesMejoresJuegos.append(valoracion[0][0])

In [47]:
print("Top 10 lista UsuarioNuevo \n \n" )
for i in range(0,len(posicionesMejoresJuegos)):
    print(items.ix[posicionesMejoresJuegos[i]+1].title)
    #items.ix[items.ix[posicionesMejoresJuegos[i]+1].name].title
    #print(str(i+1) + " "  + items.ix[posicionesMejoresJuegos[i]+1].name) 

Top 10 lista UsuarioNuevo 
 

Nombre_Videojuego_316
Nombre_Videojuego_287
Nombre_Videojuego_270
Nombre_Videojuego_1326
Nombre_Videojuego_214
Nombre_Videojuego_321
Nombre_Videojuego_515
Nombre_Videojuego_660
Nombre_Videojuego_491
Nombre_Videojuego_665


C:\Users\Elvis\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
